In [41]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [42]:
import os
os.chdir('D:\\CHRIST\\Boot camp\\DATA')

In [43]:
# Load the dataset
data = pd.read_csv("NSSO68.csv")

C:\Users\isrin\AppData\Local\Temp\ipykernel_22444\95208774.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("NSSO68.csv")


In [44]:
df=pd.read_csv("NSSO68.csv",encoding="Latin-1", low_memory=False)

In [45]:
df.head()

,slno,grp,Round_Centre,FSU_number,Round,Schedule_Number,Sample,Sector,state,State_Region,...,pickle_v,sauce_jam_v,Othrprocessed_v,Beveragestotal_v,foodtotal_v,foodtotal_q,state_1,Region,fruits_df_tt_v,fv_tot
0,1,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1141.492400,30.942394,GUJ,2,12.000000,154.18
1,2,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,17.500000,1244.553500,29.286153,GUJ,2,333.000000,484.95
2,3,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1050.315400,31.527046,GUJ,2,35.000000,214.84
3,4,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,33.333333,1142.591667,27.834607,GUJ,2,168.333333,302.30
4,5,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,75.000000,945.249500,27.600713,GUJ,2,15.000000,148.00


In [46]:
# Unique values in the 'State_1' column
print(data['state_1'].unique())

['GUJ' 'ORI' 'CHTSD' 'MP' 'JRKD' 'WB' 'AP' 'MH' 'D&D' 'D&NH' 'MIZ' 'TRPR'
 'MANPR' 'ASSM' 'MEG' 'NAG' 'A&N' 'PNDCRY' 'TN' 'GOA' 'KA' 'KE' 'LKSDP'
 'SKM' 'Bhr' 'UP' 'RJ' 'ARP' 'DL' 'HR' 'Pun' 'HP' 'UT' 'Chandr' 'J$K']


In [47]:
# Subset data to state assigned
subset_data = data[data['state_1'] == 'D&NH'][['foodtotal_q', 'MPCE_MRP', 'MLT', 'hhdsz', 'MPCE_URP', 'Age', 'Meals_seved_to_non_hhld_members', 'Meals_At_Home', 'Possess_ration_card', 'Education', 'No_of_Meals_per_day']]
print(subset_data)

       foodtotal_q  MPCE_MRP       MLT  hhdsz  MPCE_URP  Age  \
13496    23.025980   2830.80   31350.0      4   2392.75   44   
13497    15.517143   1500.77  108075.0      6   1098.17   30   
13498    20.760920   2513.36  108075.0      1   2215.00   25   
13499    16.267077   1138.80    8250.0      6   1058.50   35   
13500    30.388403   4078.26   79200.0      4   3652.75   30   
...            ...       ...       ...    ...       ...  ...   
78855    13.286009    656.49   13205.0      7    608.00   55   
78856    17.875412   1402.78   23580.0      8   1277.50   49   
78857    16.500424    879.76  205146.0      7    795.43   50   
78858    17.500290    986.07  205146.0      5    917.40   52   
78859    28.930667   2313.31   23580.0      3   2158.33   33   

       Meals_seved_to_non_hhld_members  Meals_At_Home  Possess_ration_card  \
13496                              NaN           60.0                  1.0   
13497                              NaN           60.0                  1.0 

In [48]:
# Check for missing values
print(subset_data['MPCE_MRP'].isna().sum())
print(subset_data['MPCE_URP'].isna().sum())
print(subset_data['Age'].isna().sum())
print(subset_data['Possess_ration_card'].isna().sum())
print(subset_data['Education'].isna().sum())
print(subset_data['MLT'].isna().sum())
print(subset_data['Meals_seved_to_non_hhld_members'].isna().sum())
print(subset_data['hhdsz'].isna().sum())


0
0
0
0
0
0
24
0


In [49]:
# Impute missing values with mean
def impute_with_mean(df, columns):
    for col in columns:
        df[col].fillna(df[col].mean(), inplace=True)
    return df

In [50]:
# Columns to impute
columns_to_impute = ['Meals_seved_to_non_hhld_members']

In [51]:
# Impute missing values with mean
subset_data = impute_with_mean(subset_data, columns_to_impute)

In [52]:
# Check if imputation was successful
print(subset_data['Meals_seved_to_non_hhld_members'].isna().sum())

0


In [53]:
# Fit the regression model
X = subset_data[['MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education']]
y = subset_data['foodtotal_q']
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [54]:
# Print the regression results
print(model.coef_)
print(model.intercept_)

[ 4.72615999e-03 -9.85124497e-05  5.09191679e-03  3.24538880e-01
  2.10147867e+00 -3.44715091e-02]
-11.928952914164825


In [55]:
# Check for multicollinearity using Variance Inflation Factor (VIF)
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['features'] = X.columns
print(vif)

         VIF             features
0   9.769406             MPCE_MRP
1   4.307667             MPCE_URP
2  13.071138                  Age
3  22.565816        Meals_At_Home
4   8.208124  Possess_ration_card
5   6.398299            Education


In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Define the feature and target variables
X = subset_data[['MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education']]
y = subset_data['foodtotal_q']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Add a constant to the model (intercept)
X_train_sm = sm.add_constant(X_train)

# Create a statsmodels OLS regression model
model = sm.OLS(y_train, X_train_sm).fit()

# Get the summary of the model
summary = model.summary()
print(summary)


                            OLS Regression Results                            
Dep. Variable:            foodtotal_q   R-squared:                       0.815
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     107.2
Date:                Sun, 23 Jun 2024   Prob (F-statistic):           5.84e-51
Time:                        20:39:32   Log-Likelihood:                -390.80
No. Observations:                 153   AIC:                             795.6
Df Residuals:                     146   BIC:                             816.8
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -7.5331    

In [57]:
# Construct the equation
equation = "y = " + str(round(model.params[0], 9))
for i, coef in enumerate(model.params[1:]):
    equation += " + " + str(round(coef, 9)) + "*x" + str(i)
print("Equation:", equation)

Equation: y = -7.533132392 + 0.005226757*x0 + -0.000241361*x1 + 0.002336219*x2 + 0.25031637*x3 + 1.998909915*x4 + -0.089175012*x5


In [40]:
# Print the first row of each column
print(subset_data.head(1))

       foodtotal_q  MPCE_MRP      MLT  hhdsz  MPCE_URP  Age  \
13496     23.02598    2830.8  31350.0      4   2392.75   44   

       Meals_seved_to_non_hhld_members  Meals_At_Home  Possess_ration_card  \
13496                         1.220238           60.0                  1.0   

       Education  No_of_Meals_per_day  
13496        7.0                  2.0  
